In [4]:
# 빅카인즈 뉴스 정보 수집하기
#Step 1. 필요한 모듈을 로딩합니다
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys       
import numpy           
import pandas  as pd    
import math

#Step 2. 필요한 정보를 수집합니다

query_txt = input('1.크롤링할 뉴스의 키워드는 무엇입니까?: ')
start_date = input('2.조회를 시작할 날짜를 입력하세요(예:2017-01-01) :')
end_date = input('3.조회를 종료할 날짜를 입력하세요(예:2017-12-31): ')
f_name = input('4.xls 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: ')

#Step 3. 크롬 드라이버 설정하고 빅카인즈 페이지를 엽니다
path = "C:/Users/wowog/Desktop/새 폴더 (10)/awd/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://www.bigkinds.or.kr/')
time.sleep(2)

s_time = time.time( )     

# 팝업창이 있을 경우 닫기
#driver.find_element_by_xpath(''' //*[@id="contents"]/div[3]/div/div[2]/div/div[2]/button''').click()

#Step 4. 검색 조건 입력하기
# 시작 날짜 입력하기
driver.find_element_by_xpath('//*[@id="news-search-form"]/div/div[1]/button').click()
s_date = driver.find_element_by_xpath("""//*[@id="news-search-form"]/div/div[1]/div[2]/div/div[1]/div[1]/a""")   
time.sleep(1)
driver.find_element_by_xpath("""//*[@id="search-begin-date"]""").click()
s_date.clear( )  
time.sleep(1)
for c in start_date:
    s_date.send_keys(c)
    time.sleep(0.3)
driver.find_element_by_xpath("""//*[@id="date-filter-div"]/div""").click()
time.sleep(1)
s_date.send_keys("\n")

# 종료 날짜 입력하기    
e_date = driver.find_element_by_xpath("""//*[@id="search-end-date"]""")
time.sleep(1)
driver.find_element_by_xpath("""//*[@id="search-end-date"]""").click()
e_date.clear()
time.sleep(1)
for c in end_date:
    e_date.send_keys(c)
    time.sleep(0.3)    
time.sleep(1)
e_date.send_keys("\n")

driver.find_element_by_id('date-confirm-btn').click()

# 검색어 입력한 후 검색하기    
element = driver.find_element_by_id("total-search-key")
driver.find_element_by_id("total-search-key").click( )
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(1)

# 100건씩 검색하기 누르기
#driver.find_element_by_xpath("""//*[@id="collapse-step-2"]/div/div/div[2]/div[2]/div/div[2]/div[2]/select""").click()
#element1 = Select(driver.find_element_by_xpath('//*[@id="collapse-step-2"]/div/div/div[2]/div[2]/div/div[2]/div[2]/select'))
#element1.select_by_value('100')

# Step 5. 크롤링 건수 추출 후 건수 입력받기
time.sleep(10)
driver.find_element_by_xpath("""//*[@id="collapse-step-2"]/div/div/div[2]/div[2]/div/div[2]/div[2]/select""").click()
driver.find_element_by_xpath("""//*[@id="collapse-step-2"]/div/div/div[2]/div[2]/div/div[2]/div[2]/select/option[4]""").click()

time.sleep(3)
total2 = driver.find_element_by_xpath("""//*[@id="total-news-cnt"]""").text
total = total2.replace(",","")
print("검색된 기사의 개수는 총 %s 건 입니다"%total)
cnt = int(input(' %s 건의 기사중 몇 건을 수집할까요? :' %total))

if cnt =='' :
    cnt = total

page_cnt = math.ceil(cnt / 100) 

#Step 6. 뉴스 내용 추출하기

no2=[]           # 게시글 번호 컬럼
title2=[ ]       # 게시글 제목 컬럼
dat2=[ ]        # 게시글 날짜 컬럼
source2=[ ]      #게시글 출처 컬럼
cont2=[ ]
no = 1           # 게시글 번호 초기값

def scroll_down(driver):
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        #driver.execute_script("window.scrollBy(0,9300);")
        time.sleep(2)

# 페이지 번호 만들기
page_no=[]

for i in range(7,cnt) :
    if i % 7 == 0 :
            page_no.append(i + 1)


for x in range(1,page_cnt+1) :
    print("\n")
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    news_list = soup.select('div.news-item__body')
    #news_no = 1

    for i in news_list:

        # 1.게시글 번호
        print("\n")
        print("%s 번째 뉴스 정보를 추출하고 있습니다============" %no)
        no2.append(no)
        print("1.번호 : %s" %no)

        # 2. 게시글 제목
        title=i.find('h4','news-item__title news-detail').get_text()
        title2.append(title.strip())
        print("2.제목 : %s" %title.strip())

        # 3. 게시 날짜
        dat = i.find('div','news-item__meta').find('span','news-item__date').get_text().strip()
        dat2.append(dat)
        print("3.게시날짜 : %s" %dat)

        # 4.게시글 출처
        source=i.find('div', 'news-item__meta').get_text().replace("\n","").strip()
        source1 = source.split(' ')
        source2.append(source1[0].strip())
        print("4.출처 : %s" %source1[0].strip())

        #5.게시글 내용        
        # 원래 페이지에서 게시글 제목을 눌러 게시글 상세페이지로 이동
        j=int(news_list.index(i))+1
        driver.find_element_by_xpath("""//*[@id="news-results"]/div[%s]/div[2]/div[1]"""%j).click()

        time.sleep(2)

        scroll_down(driver)

        html_1 = driver.page_source
        soup_1 = BeautifulSoup(html_1, 'html.parser')     
        cont=soup_1.find("div","news-detail__content").get_text().replace("\n","").strip()
        cont2.append(cont)
        print("5.내용 : %s" %cont)

        time.sleep(1)

        # 기사 본문 아래의 닫기 버튼 누르기
        driver.find_element_by_xpath("""//*[@id="news-detail-modal"]/div/div/div[3]/button""").click()

        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)

        #news_no += 1
        no += 1

        if no > cnt :
            break

    p = x + 1

    k = math.floor(x/7) -1

    if no >= cnt :
        break
    elif p == page_no[k]:
        driver.find_element_by_xpath('//*[@id="news-results-pagination"]/ul/li[10]/a').click()
        time.sleep(5)
    else :
        driver.find_element_by_link_text('%s' %p).click() # 다음 페이지번호 클릭
        time.sleep(10)
        print("%s 페이지로 이동합니다 " %p)


# Step 7. 출력 결과 저장하기
newsall = pd.DataFrame()
newsall['번호']=no2
newsall['제목']=pd.Series(title2)
newsall['날짜']=pd.Series(dat2)
newsall['출처']=pd.Series(source2)
newsall['내용']=pd.Series(cont2)


# 엑셀 형태로 저장하기
newsall.to_excel(f_name,index=False , engine='xlsxwriter')

e_time = time.time( )     
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집 완료 했습니다" %(cnt,no-1))
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: xls 파일명 : %s " %f_name)
print("=" *80)

driver.close( )


1.크롤링할 뉴스의 키워드는 무엇입니까?: 전주시 자전거도로
2.조회를 시작할 날짜를 입력하세요(예:2017-01-01) :2016-01-01
3.조회를 종료할 날짜를 입력하세요(예:2017-12-31): 2021-05-09
4.xls 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: C:/Users/wowog/Desktop/그래프/크롤링/data.xls


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=90.0.4430.93)
